In [1]:
#Libraries

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import os
import re
import json
import string
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
from tqdm.autonotebook import tqdm
from functools import partial
import torch
import random
from sklearn.model_selection import train_test_split
#!pip install transformers
#from transformers import BertTokenizer, BertModel
#import spacy

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

print(f'GPU available: {torch.cuda.is_available()}')
random.seed(10)

Thu Apr 29 17:57:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
print(torch.cuda.is_available())
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print("Using device:", device)

True
Using device: cuda


In [4]:
#data = pd.read_csv("data.csv")
data = pd.read_csv("data.csv")
data.head()
print(len(data))

4155


In [ ]:
def read_GloVe(filename):
 embeddings = {}
 for line in open(filename).readlines():

    fields = line.strip().split(" ")
    word = fields[0]
    embeddings[word] = [float(x) for x in fields[1:]]
 return embeddings

GloVe = read_GloVe("glove.840B.300d.conll_filtered.txt")

In [ ]:
data.head()

,Unnamed: 0,Id,pub_title,dataset_title,dataset_label,cleaned_label,text,mask
0,4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This article investigates an important factor...,True
1,5,81b5f874-9b42-4d4f-8c50-b4bab24f8aed,The Gender Gap Reloaded: Are School Characteri...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This study examines the wage gender gap of yo...,True
2,6,f35fdfa6-a3f9-4ef3-a858-c8e2efcd2349,The High School Environment and the Gender Gap...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Despite the striking reversal of the gender g...,True
3,8,e5a4c6af-0854-4f7e-b088-06f03c18a01a,Locus of Control and Peer Relationships Among ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Abstract Past research has shown that locus o...,True
4,12,d5ff0ecd-ccf5-4f8a-a09c-182376eed4d0,Beyond the Laboratory: Evaluating the Survey E...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,"The black-white gap in achievement, as measur...",True


In [ ]:
import re
paragraphs = []
copy = []
for i in range(len(data)):
  t = data.loc[i][6]
  t = t.lower()
  t = re.findall(r"[\w']+|[.,!?;]", t)
  paragraphs.append(t)
  copy.append((t, data.loc[i][5]))


In [ ]:
#Create mappings between tokens and indices.

from collections import Counter
import random

#Will need this later to remove 50% of words that only appear once in the training data from the vocabulary (and don't have GloVe embeddings).
wordCounts = Counter([w for l in paragraphs for w in l])
singletons = set([w for (w,c) in wordCounts.items() if c == 1 and not w in GloVe.keys()])


#Build dictionaries to map from words, characters to indices and vice versa.
#Save first two words in the vocabulary for padding and "UNK" token.
word2i = {w:i+2 for i,w in enumerate(set([w for l in paragraphs for w in l] + list(GloVe.keys())))}

i2word = {i:w for w,i in word2i.items()}




#When training, randomly replace singletons with UNK tokens sometimes to simulate situation at test time.
def getDictionaryRandomUnk(w, dictionary, train=False):
  if train and (w in singletons and random.random() > 0.5):
    return 1
  else:
    return dictionary.get(w, 1)

#Map a list of sentences from words to indices.
def sentences2indices(words, dictionary, train=False):
  #1.0 => UNK
  return [[getDictionaryRandomUnk(w,dictionary, train=train) for w in l] for l in words]

X       = sentences2indices(paragraphs, word2i, train=True)


print(i2word[253])

#Print out some examples of what the dev inputs will look like
for i in range(10):
  print(" ".join([i2word.get(w,'UNK') for w in X[i]]))
def prepare_input(X_list):
  X_padded = torch.nn.utils.rnn.pad_sequence([torch.as_tensor(l) for l in X_list], batch_first=True).type(torch.LongTensor)
  X_mask   = torch.nn.utils.rnn.pad_sequence([torch.as_tensor([1.0] * len(l)) for l in X_list], batch_first=True).type(torch.FloatTensor)
  return (X_padded, X_mask)

dpi
this article investigates an important factor in student achievement parental involvement . using data from the national education longitudinal study nels , we estimate a value added education production function that includes parental effort as an input . parental effort equations are also estimated as a function of child , parent , household , and school characteristics . our results suggest that parental effort has a strong positive effect on achievement that is large relative to the effect of school resources and is not captured by family background variables . parents appear to reduce their effort in response to increased school resources , suggesting potential ''crowding out'' of school resources .
this study examines the wage gender gap of young adults in the 1970s , 1980s , and 2000 in the us . using quantile regression we estimate the gender gap across the entire wage distribution . we also study the importance of high school characteristics in predicting future labor mark

In [ ]:
import torch.nn as nn
num_words = len(word2i) + 2
weights = torch.Tensor(np.zeros((num_words, 300)))
E = nn.Embedding(num_words, 300)
for i in range(0,num_words):
  if i in i2word.keys():
    if i2word[i] in GloVe.keys():
      weights[i] = torch.Tensor(GloVe[i2word[i]])
    else:
      
      weights[i] = E.weight[i]
  else:
    weights[i] = E.weight[i]
E.weight.data = weights



In [ ]:
gold_seq = []
count = 0
for i in range(len(data)):
  words = paragraphs[i]
  label = data.loc[i][4]
  t = label.lower()
  t = re.findall(r"[\w']+|[.,!?;]", t)
  seq = np.zeros(len(words))
  for j in range(len(words)):
    if words[j:j+len(t)] == t:
      index = (j, j+len(t))
      break
    else:
      index = False
  seq = np.zeros(len(words))
  if index:
    for j in range(index[0], index[1]):
      seq[j] = 1
  if all(v == 0 for v in seq):
    count+= 1
    for j in range(len(words)):
      for z in t:
        if words[j] in z:
          seq[j] = 1
          t.remove(z)
          break   
  gold_seq.append(seq)

    



In [ ]:
copy, test_x, gold_seq, test_y = train_test_split(copy, gold_seq, test_size=0.3, random_state=10)

In [ ]:
paragraphs = []
for i in copy:
  paragraphs.append(i[0])
  


In [ ]:

class BasicLSTMtagger(nn.Module):
    def __init__(self, DIM_EMB=10, DIM_HID=10):
        super(BasicLSTMtagger, self).__init__()

        self.E = nn.Embedding(num_words, 300)
        self.init_glove(GloVe)
        self.L = nn.LSTM(DIM_EMB, DIM_HID, bidirectional=True, batch_first=True)
        self.W = nn.Linear(DIM_HID*2, 2)
        self.logSoftmax = nn.LogSoftmax(dim=2)
        #TODO: initialize parameters - embedding layer, nn.LSTM, nn.Linear and nn.LogSoftmax

    def forward(self, X, train=False):
        #TODO: Implement the forward computation.
        X = X.cuda()
        
        x = self.E(X)

        x, hidden = self.L(x)

        x = self.W(x)
        return self.logSoftmax(x)  

    def init_glove(self, GloVe):
      weights = torch.Tensor(np.zeros((num_words, 300)))
      for i in range(0,num_words):
        if i in i2word.keys():
          if i2word[i] in GloVe.keys():
            weights[i] = torch.Tensor(GloVe[i2word[i]])
          else:
            
            weights[i] = self.E.weight[i]
        else:
          weights[i] = self.E.weight[i]
      self.E.weight.data = weights
    def inference(self, sentences):

      pred = self.forward(sentences).argmax(dim=2)
      return pred


#The following code will initialize a model and test that your forward computation runs without errors.
lstm   = BasicLSTMtagger(DIM_HID=7, DIM_EMB=300)
lstm = lstm.cuda()
print(X[0])
lstm_output = lstm.forward(torch.LongTensor(X[0]).reshape((1,len(X[0])))).cuda()


#Check the shape of the lstm_output and one-hot label tensors.



[22304, 70801, 10987, 46533, 49881, 8148, 28775, 46650, 22371, 34336, 48116, 115, 71540, 50692, 8676, 1450, 27121, 66660, 5725, 71325, 20336, 29803, 41077, 18785, 33494, 40908, 6808, 66660, 17576, 36377, 46344, 48878, 34336, 60169, 61498, 46533, 59185, 115, 34336, 60169, 60864, 60502, 12370, 17112, 61498, 33494, 36377, 30349, 66645, 29803, 12217, 29803, 41769, 29803, 69392, 6061, 63323, 115, 33001, 67624, 39832, 46344, 34336, 60169, 32165, 33494, 3870, 65359, 29084, 35443, 22371, 46344, 35547, 46207, 20583, 64832, 1450, 29084, 30349, 6061, 27078, 69392, 35547, 12034, 51590, 42516, 17779, 11402, 36810, 115, 26607, 61234, 64832, 54159, 18669, 60169, 28775, 34969, 64832, 53527, 6061, 27078, 29803, 37326, 56813, 37033, 22120, 30349, 6061, 27078, 115]


In [ ]:
#Training

from random import sample
import tqdm
import os
import subprocess
import random
import torch.optim as optim
def shuffle_sentences(sentences, tags):
  shuffled_sentences = []
  shuffled_tags      = []
  indices = list(range(len(sentences)))
  random.shuffle(indices)
  for i in indices:
    #print(len(sentences[i]), len(tags[i]))
    shuffled_sentences.append(sentences[i])
    shuffled_tags.append(tags[i])
  return (shuffled_sentences, shuffled_tags)
nEpochs = 7

def train_basic_lstm(sentences, tags, lstm):
  optimizer = optim.Adadelta(lstm.parameters(), lr=0.1)
  #TODO: initialize optimizer
  
  batchSize = 10
  for epoch in range(nEpochs):
    totalLoss = 0.0
    l = nn.CrossEntropyLoss(reduction="sum", ignore_index=10).cuda()
    (shuffled_sentences, shuffled_tags) = shuffle_sentences(sentences, tags)
    X       = sentences2indices(shuffled_sentences, word2i, train=True)
    for batch in tqdm.notebook.tqdm(range(0, len(sentences) - batchSize, batchSize), leave=False):
      lstm.zero_grad()
      #TODO: Impelement gradient update.
      
      input = prepare_input(X[batch:batch+batchSize-1])[0]
      Y = []
      for j in range(batch, batch+batchSize-1):
        Y.append(torch.LongTensor(shuffled_tags[j]))
      Y_padded = torch.nn.utils.rnn.pad_sequence(Y, batch_first=True,padding_value=10)
      lstm_output = lstm.forward(input)
      loss = l(torch.flatten(lstm_output, start_dim=0, end_dim = 1), torch.flatten(Y_padded).cuda())
      loss.backward()
      totalLoss += loss
      
      optimizer.step()
      torch.cuda.empty_cache()

    print(f"loss on epoch {epoch} = {totalLoss}")
    
lstm = BasicLSTMtagger(DIM_HID=500, DIM_EMB=300).cuda()
train_basic_lstm(paragraphs, gold_seq, lstm)

loss on epoch 0 = 61518.8125


loss on epoch 1 = 14917.763671875


loss on epoch 2 = 12598.5439453125


loss on epoch 3 = 11577.103515625


loss on epoch 4 = 10791.9296875


loss on epoch 5 = 9637.771484375


loss on epoch 6 = 8817.748046875


In [ ]:
torch.save(lstm.state_dict(), "lstm.pt")

In [ ]:
for j in range(0,len(data), 50):
  z = torch.LongTensor(X[j])

  result = lstm.inference(z.reshape((1,len(X[j])))).tolist()
  result = np.array(result[0])
  index = np.where(result == 1)[0]
  print(index)
  for i in index:
    print(i2word[X[j][i]])
  print(data.loc[j])


[16 17 18 19]
national
education
longitudinal
study
Unnamed: 0                                                       4
Id                            c754dec7-c5a3-4337-9892-c02158475064
pub_title        Parental Effort, School Resources, and Student...
dataset_title                National Education Longitudinal Study
dataset_label                National Education Longitudinal Study
cleaned_label                national education longitudinal study
text              This article investigates an important factor...
mask                                                          True
Name: 0, dtype: object
[ 817  905  906 1090 1091]
slosh
slosh
model
slosh
model
Unnamed: 0                                                     347
Id                            07ce992d-e2f3-49cb-bedb-49616a19405f
pub_title        U.S. IOOS coastal and ocean modeling testbed: ...
dataset_title       Sea, Lake, and Overland Surges from Hurricanes
dataset_label                                          SLOSH mod

KeyboardInterrupt: ignored

In [ ]:
def jaccard(str1, str2): 
    a = set(str1) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))
x = []
y = []
for i in test_x:
  x.append(i[0])
  y.append(i[1])
X       = sentences2indices(x, word2i, train=True)
score = 0
for j in range(0,len(X)):
  z = torch.LongTensor(X[j])

  result = lstm.inference(z.reshape((1,len(X[j])))).tolist()
  result = np.array(result[0])
  index = np.where(result == 1)[0]
  a = []
  for i in index:
    a.append(i2word[X[j][i]])
  if jaccard(a, y[j]) >= 0.5:
    score += 1
print(score/len(X))



0.7249398556535686


In [ ]:
x = []
y = []
for i in copy:
  x.append(i[0])
  y.append(i[1])
X       = sentences2indices(x, word2i, train=True)
score = 0
for j in range(0,len(X)):
  z = torch.LongTensor(X[j])

  result = lstm.inference(z.reshape((1,len(X[j])))).tolist()
  result = np.array(result[0])
  index = np.where(result == 1)[0]
  a = []
  for i in index:
    a.append(i2word[X[j][i]])
  if jaccard(a, y[j]) >= 0.5:
    score += 1
print(score/len(X))

0.7455295735900963


In [ ]:
from google.colab import files
files.download("lstm.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
text = []
for i in range(len(data)):
  t = data.loc[i][6]


  text.append((t, data.loc[i][5]))


In [6]:
pad_word = "<pad>"
bos_word = "<s>"
eos_word = "</s>"
unk_word = "<unk>"
pad_id = 0
bos_id = 1
eos_id = 2
unk_id = 3
    
def normalize_sentence(s):
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

class Vocabulary:
    def __init__(self):
        self.word_to_id = {pad_word: pad_id, bos_word: bos_id, eos_word:eos_id, unk_word: unk_id}
        self.word_count = {}
        self.id_to_word = {pad_id: pad_word, bos_id: bos_word, eos_id: eos_word, unk_id: unk_word}
        self.num_words = 4
    
    def get_ids_from_sentence(self, sentence):
        sentence = normalize_sentence(sentence)
        sent_ids = [bos_id] + [self.word_to_id[word] if word in self.word_to_id \
                               else unk_id for word in sentence.split()] + \
                               [eos_id]
        return sent_ids
    
    def tokenized_sentence(self, sentence):
        sent_ids = self.get_ids_from_sentence(sentence)
        return [self.id_to_word[word_id] for word_id in sent_ids]

    def decode_sentence_from_ids(self, sent_ids):
        words = list()
        for i, word_id in enumerate(sent_ids):
            if word_id in [bos_id, eos_id, pad_id]:
                # Skip these words
                continue
            else:
                words.append(self.id_to_word[word_id])
        return ' '.join(words)

    def add_words_from_sentence(self, sentence):
        sentence = normalize_sentence(sentence)
        for word in sentence.split():
            if word not in self.word_to_id:
                # add this word to the vocabulary
                self.word_to_id[word] = self.num_words
                self.id_to_word[self.num_words] = word
                self.word_count[word] = 1
                self.num_words += 1
            else:
                # update the word count
                self.word_count[word] += 1

vocab = Vocabulary()
for src, tgt in text:
    vocab.add_words_from_sentence(src)
    vocab.add_words_from_sentence(tgt)
print(f"Total words in the vocabulary = {vocab.num_words}")

Total words in the vocabulary = 56347


Seq2Seq

In [ ]:
for src, tgt in text[:3]:
    sentence = src
    word_tokens = vocab.tokenized_sentence(sentence)
    # Automatically adds bos_id and eos_id before and after sentence ids respectively
    word_ids = vocab.get_ids_from_sentence(sentence)
    print(sentence)
    print(word_tokens)
    print(word_ids)
    print(vocab.decode_sentence_from_ids(word_ids))

 Introduction: Characterizing progression in Alzheimer's disease is critically important for early detection and targeted treatment. The objective was to develop a prognostic model, based on multivariate longitudinal markers, for predicting progression-free survival in patients with mild cognitive impairment. Methods: The information contained in multiple longitudinal markers was extracted using multivariate functional principal components analysis and used as predictors in the Cox regression models. Cross-validation was used for selecting the best model based on Alzheimer's Disease Neuroimaging Initiative-1. External validation was conducted on Alzheimer's Disease Neuroimaging Initiative-2. Results: Model comparison yielded a prognostic index computed as the weighted combination of historical information of five neurocognitive longitudinal markers that are routinely collected in observational studies. The comprehensive validity analysis provided solid evidence of the usefulness of the

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import tqdm
import nltk
from google.colab import files
class SingleTurnMovieDialog_dataset(Dataset):
    """Single-Turn version of Cornell Movie Dialog Cropus dataset."""

    def __init__(self, conversations, vocab, device):
        """
        Args:
            conversations: list of tuple (src_string, tgt_string) 
                         - src_string: String of the source sentence
                         - tgt_string: String of the target sentence
            vocab: Vocabulary object that contains the mapping of 
                    words to indices
            device: cpu or cuda
        """
        self.conversations = conversations
        self.vocab = vocab
        self.device = device

        def encode(src, tgt):
            src_ids = self.vocab.get_ids_from_sentence(src)
            tgt_ids = self.vocab.get_ids_from_sentence(tgt)
            return (src_ids, tgt_ids)

        # We will pre-tokenize the conversations and save in id lists for later use
        self.tokenized_conversations = [encode(src, tgt) for src, tgt in self.conversations]
        
    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        return {"conv_ids":self.tokenized_conversations[idx], "conv":self.conversations[idx]}

def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (src_seq, trg_seq).
    We should build a custom collate_fn rather than using default collate_fn,
    because merging sequences (including padding) is not supported in default.
    Seqeuences are padded to the maximum length of mini-batch sequences (dynamic padding).
    Args:
        data: list of dicts {"conv_ids":(src_ids, tgt_ids), "conv":(src_str, trg_str)}.
            - src_ids: list of src piece ids; variable length.
            - tgt_ids: list of tgt piece ids; variable length.
            - src_str: String of src
            - tgt_str: String of tgt
    Returns: dict { "conv_ids":     (src_ids, tgt_ids), 
                    "conv":         (src_str, tgt_str), 
                    "conv_tensors": (src_seqs, tgt_seqs)}
            src_seqs: torch tensor of shape (src_padded_length, batch_size).
            trg_seqs: torch tensor of shape (tgt_padded_length, batch_size).
            src_padded_length = length of the longest src sequence from src_ids
            tgt_padded_length = length of the longest tgt sequence from tgt_ids
    """
    # Sort conv_ids based on decreasing order of the src_lengths.
    # This is required for efficient GPU computations.
    src_ids = [torch.LongTensor(e["conv_ids"][0]) for e in data]
    tgt_ids = [torch.LongTensor(e["conv_ids"][1]) for e in data]
    src_str = [e["conv"][0] for e in data]
    tgt_str = [e["conv"][1] for e in data]
    data = list(zip(src_ids, tgt_ids, src_str, tgt_str))
    data.sort(key=lambda x: len(x[0]), reverse=True)
    src_ids, tgt_ids, src_str, tgt_str = zip(*data)
    src_seqs = nn.utils.rnn.pad_sequence(src_ids, padding_value=pad_id)
    tgt_seqs = nn.utils.rnn.pad_sequence(tgt_ids, padding_value=pad_id)

    # Pad the src_ids and tgt_ids using token pad_id to create src_seqs and tgt_seqs
    
    # Implementation tip: You can use the nn.utils.rnn.pad_sequence utility
    # function to combine a list of variable-length sequences with padding.
    
    # YOUR CODE HERE
    ...

    return {"conv_ids":(src_ids, tgt_ids), "conv":(src_str, tgt_str), "conv_tensors":(src_seqs.to(device), tgt_seqs.to(device))}

In [12]:
from sklearn.model_selection import train_test_split
import random
random.seed(10)
text, test = train_test_split(text, test_size=0.3, random_state=10)

In [ ]:
dataset = SingleTurnMovieDialog_dataset(text, vocab, device)

batch_size = 5

data_loader = DataLoader(dataset=dataset, batch_size=batch_size, 
                               shuffle=True, collate_fn=collate_fn)

In [8]:
import torch.nn as nn
class Seq2seqBaseline(nn.Module):
    def __init__(self, vocab, emb_dim = 300, hidden_dim = 300, num_layers = 2, dropout=0.1):
        super().__init__()

        # Initialize your model's parameters here. To get started, we suggest
        # setting all embedding and hidden dimensions to 300, using encoder and
        # decoder GRUs with 2 layers, and using a dropout rate of 0.1.

        # Implementation tip: To create a bidirectional GRU, you don't need to
        # create two GRU networks. Instead use nn.GRU(..., bidirectional=True).
        
        self.num_words = vocab.num_words
        self.emb_dim = emb_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.E = nn.Embedding(self.num_words, emb_dim)

        self.encoder = nn.GRU(emb_dim, hidden_dim, 2, dropout=dropout, bidirectional=True)
        self.decoder = nn.GRU(emb_dim, hidden_dim, 2, dropout=dropout, bidirectional=False)
        self.lin = nn.Linear(emb_dim, vocab.num_words)
        # YOUR CODE HERE

    def encode(self, source):
        """Encode the source batch using a bidirectional GRU encoder.

        Args:
            source: An integer tensor with shape (max_src_sequence_length,
                batch_size) containing subword indices for the source sentences.

        Returns:
            A tuple with three elements:
                encoder_output: The output hidden representation of the encoder 
                    with shape (max_src_sequence_length, batch_size, hidden_size).
                    Can be obtained by adding the hidden representations of both 
                    directions of the encoder bidirectional GRU. 
                encoder_mask: A boolean tensor with shape (max_src_sequence_length,
                    batch_size) indicating which encoder outputs correspond to padding
                    tokens. Its elements should be True at positions corresponding to
                    padding tokens and False elsewhere.
                encoder_hidden: The final hidden states of the bidirectional GRU 
                    (after a suitable projection) that will be used to initialize 
                    the decoder. This should be a tensor h_n with shape 
                    (num_layers, batch_size, hidden_size). Note that the hidden 
                    state returned by the bi-GRU cannot be used directly. Its 
                    initial dimension is twice the required size because it 
                    contains state from two directions.

        The first two return values are not required for the baseline model and will
        only be used later in the attention model. If desired, they can be replaced
        with None for the initial implementation.
        """

        # Implementation tip: consider using packed sequences to more easily work
        # with the variable-length sequences represented by the source tensor.
        # See https://pytorch.org/docs/stable/nn.html#torch.nn.utils.rnn.PackedSequence.

        # https://stackoverflow.com/questions/51030782/why-do-we-pack-the-sequences-in-pytorch

        # Implementation tip: there are many simple ways to combine the forward
        # and backward portions of the final hidden state, e.g. addition, averaging,
        # or a linear transformation of the appropriate size. Any of these
        # should let you reach the required performance.

        # Compute a tensor containing the length of each source sequence.

        hid = torch.zeros(4, source.shape[1], 300).cuda()
        encoder_mask = source != 0
        x = self.E(source)
        x, hid = self.encoder(x, hid)

        x = x[:,:,:self.hidden_dim] + x [:,:,self.hidden_dim:]
        hid = hid[:self.num_layers,:,:] + hid[self.num_layers:,:,:]

        # YOUR CODE HERE
        ...
        return  x, encoder_mask, hid

    def decode(self, decoder_input, last_hidden, encoder_output, encoder_mask):
        """Run the decoder GRU for one decoding step from the last hidden state.

        The third and fourth arguments are not used in the baseline model, but are
        included for compatibility with the attention model in the next section.

        Args:
            decoder_input: An integer tensor with shape (1, batch_size) containing 
                the subword indices for the current decoder input.
            last_hidden: A pair of tensors h_{t-1} representing the last hidden
                state of the decoder, each with shape (num_layers, batch_size,
                hidden_size). For the first decoding step the last_hidden will be 
                encoder's final hidden representation.
            encoder_output: The output of the encoder with shape
                (max_src_sequence_length, batch_size, hidden_size).
            encoder_mask: The output mask from the encoder with shape
                (max_src_sequence_length, batch_size). Encoder outputs at positions
                with a True value correspond to padding tokens and should be ignored.

        Returns:
            A tuple with three elements:
                logits: A tensor with shape (batch_size,
                    vocab_size) containing unnormalized scores for the next-word
                    predictions at each position.
                decoder_hidden: tensor h_n with the same shape as last_hidden 
                    representing the updated decoder state after processing the 
                    decoder input.
                attention_weights: This will be implemented later in the attention
                    model, but in order to maintain compatible type signatures, we also
                    include it here. This can be None or any other placeholder value.
        """

        # These arguments are not used in the baseline model.
        del encoder_output
        del encoder_mask
        decoder_input = torch.unsqueeze(decoder_input, 0)
        x = self.E(decoder_input)
        x, hid = self.decoder(x, last_hidden)
        x = self.lin(torch.squeeze(x, 0))


        return x, hid, 0
        # YOUR CODE HERE
        ...

    def compute_loss(self, source, target):
        """Run the model on the source and compute the loss on the target.

        Args:
            source: An integer tensor with shape (max_source_sequence_length,
                batch_size) containing subword indices for the source sentences.
            target: An integer tensor with shape (max_target_sequence_length,
                batch_size) containing subword indices for the target sentences.

        Returns:
            A scalar float tensor representing cross-entropy loss on the current batch
            divided by the number of target tokens in the batch.
            Many of the target tokens will be pad tokens. You should mask the loss 
            from these tokens using appropriate mask on the target tokens loss.
        """

        # Implementation tip: don't feed the target tensor directly to the decoder.
        # To see why, note that for a target sequence like <s> A B C </s>, you would
        # want to run the decoder on the prefix <s> A B C and have it predict the
        # suffix A B C </s>.
        x, mask, hid = self.encode(source)

        loss_func = nn.CrossEntropyLoss(ignore_index=0)
        loss = 0
        for i in range(target.shape[0]- 1):
          out, hid, temp = self.decode(target[i], hid, x, mask)

          loss += loss_func(out, target[i + 1])
        return loss/target.shape[0]

        # You may run self.encode() on the source only once and decode the target 
        # one step at a time.

        # YOUR CODE HERE
        ...

In [ ]:
def train(model, data_loader, num_epochs, model_file, learning_rate=0.0001):
    """Train the model for given number of epochs and save the trained model in 
    the final model_file.
    """

    decoder_learning_ratio = 5.0
    
    encoder_parameter_names = ["E", "encoder"]
                               
    encoder_named_params = list(filter(lambda kv: any(key in kv[0] for key in encoder_parameter_names), model.named_parameters()))
    decoder_named_params = list(filter(lambda kv: not any(key in kv[0] for key in encoder_parameter_names), model.named_parameters()))
    encoder_params = [e[1] for e in encoder_named_params]
    decoder_params = [e[1] for e in decoder_named_params]
    optimizer = torch.optim.AdamW([{'params': encoder_params},
                {'params': decoder_params, 'lr': learning_rate * decoder_learning_ratio}], lr=learning_rate)
    
    clip = 50.0
    for epoch in tqdm.notebook.trange(num_epochs, desc="training", unit="epoch"):
        # print(f"Total training instances = {len(train_dataset)}")
        # print(f"train_data_loader = {len(train_data_loader)} {1180 > len(train_data_loader)/20}")
        with tqdm.notebook.tqdm(
                data_loader,
                desc="epoch {}".format(epoch + 1),
                unit="batch",
                total=len(data_loader)) as batch_iterator:
            model.train()
            total_loss = 0.0
            for i, batch_data in enumerate(batch_iterator, start=1):
                source, target = batch_data["conv_tensors"]
                optimizer.zero_grad()
                loss = model.compute_loss(source, target)
                total_loss += loss.item()
                loss.backward()
                # Gradient clipping before taking the step
                _ = nn.utils.clip_grad_norm_(model.parameters(), clip)
                optimizer.step()

                batch_iterator.set_postfix(mean_loss=total_loss / i, current_loss=loss.item())
    # Save the model after training         
    torch.save(model.state_dict(), model_file)

In [ ]:
# You are welcome to adjust these parameters based on your model implementation.
num_epochs = 1
batch_size = 10
# Reloading the data_loader to increase batch_size
dataset = SingleTurnMovieDialog_dataset(text[0:10], vocab, device)
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, 
                               shuffle=True, collate_fn=collate_fn)

baseline_model = Seq2seqBaseline(vocab).cuda()
train(baseline_model, data_loader, num_epochs, "baseline_model.pt")
# Download the trained model to local for future use
#files.download('baseline_model.pt')

In [9]:
baseline_model = Seq2seqBaseline(vocab).cuda()
print(vocab.num_words)
print(baseline_model.E.weight.shape)

baseline_model.load_state_dict(torch.load("baseline_model_new.pt"))

56347
torch.Size([56347, 300])


<All keys matched successfully>

In [ ]:
def predict_greedy(model, sentence, max_length=100):
    """Make predictions for the given input using greedy inference.
    
    Args:
        model: A sequence-to-sequence model.
        sentence: A input string.
        max_length: The maximum length at which to truncate outputs in order to
            avoid non-terminating inference.
    
    Returns:
        Model's predicted greedy response for the input, represented as string.
    """

    # You should make only one call to model.encode() at the start of the function, 
    # and make only one call to model.decode() per inference step.
    model.eval()
    source = torch.unsqueeze(torch.tensor(vocab.get_ids_from_sentence(sentence)).cuda(), 1)
    x, mask, hid = model.encode(source)
    start = bos_id
    sent = [start]
    i = 0
    while start != eos_id and i < 100:
          out, hid, temp = model.decode(torch.unsqueeze(torch.tensor(start).cuda(), 0), hid, x, mask)
          start = torch.argmax(out[0], 0)
          sent.append(start.item())
          i += 1
    sent = vocab.decode_sentence_from_ids(sent)
    
    return sent
print(predict_greedy(baseline_model, text[2][0]))
print(text[2][1])


baltimore longitudinal study of aging
baltimore longitudinal study of aging blsa 


In [11]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))



In [13]:
def predict_beam(model, sentence, k=3, max_length=100, thresh=-9999):
    """Make predictions for the given inputs using beam search.
    
    Args:
        model: A sequence-to-sequence model.
        sentence: An input sentence, represented as string.
        k: The size of the beam.
        max_length: The maximum length at which to truncate outputs in order to
            avoid non-terminating inference.
    
    Returns:
        A list of k beam predictions. Each element in the list should be a string
        corresponding to one of the top k predictions for the corresponding input,
        sorted in descending order by its final score.
    """

    # Implementation tip: once an eos_token has been generated for any beam, 
    # remove its subsequent predictions from that beam by adding a small negative 
    # number like -1e9 to the appropriate logits. This will ensure that the 
    # candidates are removed from the beam, as its probability will be very close
    # to 0. Using this method, uou will be able to reuse the beam of an already 
    # finished candidate

    # Implementation tip: while you are encouraged to keep your tensor dimensions
    # constant for simplicity (aside from the sequence length), some special care
    # will need to be taken on the first iteration to ensure that your beam
    # doesn't fill up with k identical copies of the same candidate.
    
    # You are welcome to tweak alpha
    alpha = 0.9
    model.eval()
    beams = []
    curr = []
    source = torch.unsqueeze(torch.tensor(vocab.get_ids_from_sentence(sentence)).cuda(), 1)
    x, mask, hid = model.encode(source)
    start = bos_id
    
    out, hid, temp = model.decode(torch.unsqueeze(torch.tensor(start).cuda(), 0), hid, x, mask)
    out = torch.log_softmax(out[0], 0)
    values, start = torch.topk(out, k, 0)
    for i in range(len(values)):
      # Each beam contains the log probs at its first index and the hidden states at its last index
      beams.append([values[i], start[i].item(), hid])
    
    generation = []
    i = 0
    while i < k:
      curr = []
      for j in beams:
          out, hid, temp = model.decode(torch.unsqueeze(torch.tensor(j[-2]).cuda(), 0), j[-1], x, mask)
          out = torch.log_softmax(out[0], 0)
          values, start = torch.topk(out, k, 0)
          for z in range(len(values)):
            temp = j.copy()
            temp[0] = values[z] + temp[0]
            temp.insert(-1, start[z].item())
            temp[-1] = hid
            curr.append(temp)
      curr = sorted(curr,reverse=True, key=lambda x: x[0])
      curr = curr[0:k - i]
      beams = []
      for j in curr:
        if j[-2] == eos_id or len(j) > 20:
          generation.append(j[:-1])
          i +=1
        else:
          beams.append(j)
    final = []
    generation = sorted(generation, reverse=True, key=lambda x: x[0]/(len(x)-1)**alpha)
    for i in generation:
      if i[0].item() > thresh:
        final.append(vocab.decode_sentence_from_ids(i[1:]))
    return final

    
    # YOUR CODE HERE
    ...

In [14]:
testing = {}
for i in range(0, len(test)):
  if test[i][0] not in testing.keys():
    predictions = predict_beam(baseline_model, test[i][0], thresh=-2)
    testing[test[i][0]] = (predictions, [test[i][1]])
  else:
    testing[test[i][0]][1].append(test[i][1])

In [15]:
tp = 0
fp = 0
fn = 0
for i in testing.values():
  predictions = i[0]
  cop = predictions.copy()
  true_pred = i[1].copy()
  check = False
  #check exact match first
  for j in predictions:
    if j in true_pred:
      tp += 1
      true_pred.remove(j)
      cop.remove(j)
  #then check rest for jaccard score
  for j in cop:
    found = False
    removal = 0
    for k in true_pred:
      if jaccard(j, k) >= 0.5:
        found = True
        removal = k
        break
    if found:
      tp += 1
      true_pred.remove(removal)
    else:
      fp += 1
  fn += len(true_pred)

In [17]:
print("testing performance")
print("micro F score")
print(fp)
print(fn)
print(tp/(tp + 1/2*(fp+fn)))
print("accuracy")
print(tp/(tp+fn))

testing performance
micro F score
952
229
0.6328877836493627
accuracy
0.8163592622293504


In [18]:
testing = {}
for i in range(0, len(text)):
  if text[i][0] not in testing.keys():
    predictions = predict_beam(baseline_model, text[i][0], thresh=-2)
    testing[text[i][0]] = (predictions, [text[i][1]])
  else:
    testing[text[i][0]][1].append(text[i][1])

In [19]:
tp = 0
fp = 0
fn = 0
for i in testing.values():
  predictions = i[0]
  cop = predictions.copy()
  true_pred = i[1].copy()
  check = False
  #check exact match first
  for j in predictions:
    if j in true_pred:
      tp += 1
      true_pred.remove(j)
      cop.remove(j)
  #then check rest for jaccard score
  for j in cop:
    found = False
    removal = 0
    for k in true_pred:
      if jaccard(j, k) >= 0.5:
        found = True
        removal = k
        break
    if found:
      tp += 1
      true_pred.remove(removal)
    else:
      fp += 1
  fn += len(true_pred)

In [20]:
print("testing performance")
print("micro F score")
print(fp)
print(fn)
print(tp/(tp + 1/2*(fp+fn)))
print("accuracy")
print(tp/(tp+fn))

testing performance
micro F score
1740
539
0.675217329343024
accuracy
0.8146492434662999


In [ ]:
score = 0
for j in range(0,len(text)):
  predictions = predict_beam(baseline_model, text[j][0])
  for i in predictions:
    if jaccard(text[j][1], i) >= 0.5:
      score += 1
      break
print("training_accuracy")
print(score/len(text))

training_accuracy
0.8803301237964236


Tried testing on unfiltered dataset

In [ ]:
import math
test_unfilter = pd.read_csv("data_false.csv")
test_unfilter = test_unfilter[test_unfilter.text.notnull()].reset_index()
text_2 = []
for i in range(len(test_unfilter)):

  t = test_unfilter.loc[i][7]

  text_2.append((t, test_unfilter.loc[i][6]))


In [ ]:

print(text_2[0])

(' Federal Reserve Bank of Richmond S1. Accounting for low enrollment and graduation rates at four-year colleges when returns to graduation are high Several studies claimed that there is underinvestment in education. Judd (2000) combines Capital Asset Pricing Model (CAPM) techniques with the indivisibility of human capital to compare the return to four-year college graduation with assets of similar risk to find an excess of return to the college investment option. Heckman, Lochner, and Todd (2008) evaluate the internal rate of return of the four-year college investment option relative to work to find that since 1960, internal rates of return have been around 10 percent or higher, depending on the cohort and different specifications of labor markets and taxes. Cunha, Heckman, and Navarro (2005), using data from NLSY/1979, extend the analysis to evaluate the internal rate of return for the marginal student, the agent with the lowest observable measures of ability who enrolls in four-year

In [ ]:
score = 0
for j in range(0,len(text_2), 50):
  predictions = predict_beam(baseline_model, text_2[j][0])
  for i in predictions:
    if jaccard(text_2[j][1], i) >= 0.5:
      score += 1
      break
print("training_accuracy")
print(score/(len(text_2)/50))

training_accuracy
0.5281361002056459
